Load new images

In [62]:
#Using new_data folder
import collections
import os
data_path = '../NewData/'
mapping = collections.defaultdict(list)

for genusName in os.listdir(data_path):

    for img in os.listdir(os.path.join(data_path, genusName)):
        mapping[genusName].append(os.path.join(data_path, genusName, img))

print(len(mapping))

10


In [48]:
import pandas as pd
import numpy as np
mapping_testing = {key: np.array(value) for key, value in mapping.items()}

Our small collected sample. Only for visualization

In [49]:
totalTestingImg = 0
for dictkeys in mapping_testing.keys():
    totalCount = len(mapping_testing[dictkeys])

    totalTestingImg += len(mapping_testing[dictkeys])

    print(f"Class Name: {dictkeys}, Number of images: {totalCount}")

print(f"Total training Images: {totalTestingImg}")

Class Name: Auricularia, Number of images: 3
Class Name: Cookeina, Number of images: 3
Class Name: Entoloma, Number of images: 3
Class Name: Geastrum, Number of images: 3
Class Name: Hygrocybe, Number of images: 2
Class Name: Marasmius, Number of images: 2
Class Name: Ophiocordyceps, Number of images: 2
Class Name: Oudemansiella, Number of images: 3
Class Name: Phallus, Number of images: 3
Class Name: Trametes, Number of images: 2
Total training Images: 26


In [50]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from torch.utils.data import TensorDataset
from torchvision import transforms
#adjust to 224 dimension (input to alexnet)
transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((224,224))])
import torchvision.models
alexnet = torchvision.models.alexnet(pretrained=True)
numClasses = len(mapping_testing)

c:\Users\eric7\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\eric7\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Same as transferLearningTensor.ipynb.  Run all images through alexnet which outputs a tensor

In [51]:
def transferLearning(mapping):
    dict = {}
    print(f"Total number of classes {numClasses}")
    print("Start Transfer Learning Section: AlexNet feature extract for each class")
    iterCount = 1

    for key, images in mapping.items():
        print(f"Class {iterCount} out of {numClasses}")
        dict[key] = []
        for img in images:
            newImg = Image.open(img)
            newImg = transform(newImg)
            feat = alexnet.features(newImg)
            dict[key].append(feat)
        iterCount += 1
    return dict

In [52]:
testing_dict = transferLearning(mapping_testing)

Total number of classes 10
Start Transfer Learning Section: AlexNet feature extract for each class
Class 1 out of 10
Class 2 out of 10
Class 3 out of 10
Class 4 out of 10
Class 5 out of 10
Class 6 out of 10
Class 7 out of 10
Class 8 out of 10
Class 9 out of 10
Class 10 out of 10


In [53]:
testDict = {}

In [54]:
#make sure we have the same mapping as previously trained model
import pickle 
with open('mapping.pkl', 'rb') as file:
    mappingDict = pickle.load(file)

testDict = collections.defaultdict(list)
for idx, keyName in mappingDict.items():
    testDict[idx] = testing_dict[keyName]

In [55]:
testingData, testingLabel = [], []

for key, values in testDict.items():
    #Make the array the size of values
    testingLabel.extend([key] * len(values))
    testingData.extend(values)
testingTensor = torch.stack(testingData)
testingLabelTensor = torch.tensor(testingLabel)
test_set = TensorDataset(testingTensor, testingLabelTensor)

In [56]:
print(f'length of test set: {len(test_set)}')

length of test set: 26


In [57]:
#save the test set (just in case)
torch.save(test_set, 'NewTestSet.pt')

In [58]:
#Same function as trainModel.ipynb
def getAccuracy(net, loader):
  correct, count = 0, 0
  net.eval()
  #no gradient modification
  with torch.no_grad():
    print("Total number of batches: ", len(loader))
    for currbatch, (i, corr) in enumerate(loader):
      count += corr.shape[0]

      correct += (torch.max(net(i), 1)[1] == corr).sum().item()

  return correct / count

Same NN structure as trainModel.ipynb

In [59]:
n = 10

class preLearnedFungaV1(nn.Module):
    def __init__(self):
        super(preLearnedFungaV1, self).__init__()
        self.name = "fungaV1"
        #added anther convolution channel before fc layers
        self.conv1 = nn.Conv2d(256, 128, 3, 1, 1)
        self.fc1 = nn.Linear(128*6*6, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, n)

        #dropout implementation
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)
        self.dropout3 = nn.Dropout(0.1)
        self.dropout4 = nn.Dropout(0.1)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = x.view(x.size(0), -1)

        #FC layers
        x = self.dropout2(x)
        x = F.relu(self.fc1(x))
        
        x = self.dropout3(x)
        x = F.relu(self.fc2(x))


        x = self.dropout4(x)
        x = self.fc3(x)
        x = x.squeeze(1)  
        return x

In [60]:
def customDataLoader(batchsize):
    testing_load = torch.utils.data.DataLoader(test_set, batch_size = batchsize)
    return testing_load

In [61]:
bestNet = preLearnedFungaV1()
#model name
def get_model_name(name, batch_size, lr, epoch):
    path = "modelStorage/model_{0}_bs{1}_lr{2}_epoch{3}".format(name, batch_size, lr, epoch)
    return path
BATCH_SIZE = 64
LEARNING_RATE = 0.0006
EPOCH = 90
model_path = get_model_name("fungaV1", batch_size=BATCH_SIZE, lr=LEARNING_RATE, epoch=EPOCH)
state = torch.load(model_path)
bestNet.load_state_dict(state)

testLoad = customDataLoader(100)

dataLoad = testLoad

bestNet.eval

accuracy = getAccuracy(bestNet, dataLoad)
print("Test Classification Accuracy: ", accuracy)

Total number of batches:  1
Test Classification Accuracy:  0.6538461538461539
